<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>


<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image_predictions.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/develop/examples/prediction_upload/image_predictions.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Image Prediction Import

* This notebook walks you through the process of uploading model predictions to a Model Run. This notebook provides an example for each supported prediction type for image assets. 

A Model Run is a container for the predictions, annotations and metrics of a specific experiment in your ML model development cycle.

**Supported annotations that can be uploaded through the SDK**

- Bounding box 
- Polygon
- Point
- Polyline 
- Classification free-text
- Classification - radio
- Classification - checklist




* Notes:
    * If you are importing more than 1,000 mask predictions at a time, consider submitting separate jobs, as they can take longer than other prediction types to import.
    * After the execution of this notebook a complete Model Run with predictions will be created in your organization. 

## Setup

In [627]:
!pip install -q 'labelbox[data]'

In [628]:
from labelbox.schema.ontology import OntologyBuilder, Tool, Classification, Option
from labelbox import Client, MALPredictionImport, LabelImport
from labelbox.data.serialization import NDJsonConverter
from labelbox.schema.media_type import MediaType
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, MaskData,
    Rectangle, Point, Line, Mask, Polygon,
    Radio, Checklist, Text,
    ClassificationAnnotation, ClassificationAnswer
)
import uuid
import numpy as np
from labelbox.schema.queue_mode import QueueMode

## Replace with your API Key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [629]:
API_KEY = None
client = Client(API_KEY)

## Supported Predictions

In [630]:
########### Radio Classification ###########

# Python annotation
radio_prediction = ClassificationAnnotation(
    name="radio_question", 
    value=Radio(answer = ClassificationAnswer(name = "second_radio_answer", confidence=0.5))
)

# NDJSON
radio_prediction_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer', 'confidence': 0.5}
} 

In [631]:
########## Nested Classifications ##########


### Radio #### 
# Python annotation 
nested_radio_prediction = ClassificationAnnotation(
    name="nested_radio_question", 
    value=Radio(answer = ClassificationAnswer(name = "first_radio_answer" , confidence=0.5)),
    classifications=[
    	ClassificationAnnotation(
        	name="sub_radio_question",
      		value=Radio(answer=ClassificationAnswer(name="first_sub_radio_answer", confidence=0.5))
    )
  ]
)


# NDJSON 
nested_radio_prediction_ndjson = {
  "name": "nested_radio_question",
  "answer": { "name": "first_radio_answer", "confidence": 0.5 },
      "classifications" : [
      {
        "name": "sub_radio_question", 
        "answer": {"name": "first_sub_radio_answer", "confidence": 0.5 }
      }
    ]
}

### Checklist #### 
## Only supported with NDJSON tools

nested_checklist_prediction_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer", 
      "confidence": 0.5,
      "classifications" : [
        {
          "name": "sub_checklist_question", 
          "answer": {"name": "first_sub_checklist_answer", "confidence": 0.5 }
        }          
      ]         
  }]
}



In [632]:
############ Checklist ############

# Python Annotations
checklist_prediction = ClassificationAnnotation(
  name="checklist_question", # must match your ontology feature's name
  value=Checklist(
      answer = [
        ClassificationAnswer(
            name = "first_checklist_answer", 
            confidence=0.5
        ), 
        ClassificationAnswer(
            name = "second_checklist_answer", 
            confidence=0.5
        )
      ]
    )
 )

# NDJSON
checklist_prediction_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer' , 'confidence': 0.5},
    {'name': 'second_checklist_answer', 'confidence': 0.5}
  ]
}

In [633]:
####### Bounding box #######


# Python Annotation 
bbox_prediction = ObjectAnnotation(
  name = "bounding_box",  # must match your ontology feature's name
  confidence=0.5, 
  value=Rectangle(
        start=Point(x=977, y=1690), # Top left
        end=Point(x=330, y=225), # Bottom right
    ),
  
)

#NDJSON 
bbox_prediction_ndjson = {
  'name': 'bounding_box', 
  'confidence': 0.5,
  'bbox': {
          "top": 977,
          "left": 1690,
          "height": 330,
          "width": 225
      }
}


In [634]:
####### Bounding box with nested classification #######
bbox_with_radio_subclass_prediction = ObjectAnnotation(
    name="bbox_with_radio_subclass",
    confidence=0.5, # must match your ontology feature's name
    value=Rectangle(
        start=Point(x=933, y=541), # Top left
        end=Point(x=191, y=330), # Bottom right
    ),
    classifications=[
    	ClassificationAnnotation(
        	name="sub_radio_question",
      		value=Radio(answer=ClassificationAnswer(name="first_sub_radio_answer", confidence=0.5))
    )
  ]
)


## NDJSON
bbox_with_radio_subclass_prediction_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "confidence": 0.5,
    "classifications": [{
        "name": "sub_radio_question",
        "confidence": 0.5,
        "answer": 
            { "name":"first_sub_radio_answer", "confidence": 0.5}
         
    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}

In [635]:
########## Polygon ##########
# Python Anotation 
polygon_prediction = ObjectAnnotation(
  name = "polygon",  # must match your ontology feature's name 
  confidence = 0.5, 
  value=Polygon( # Coordinates for the verticies of your polygon
        points=[Point(x=1489.581,y=183.934),Point(x=2278.306,y=256.885),Point(x=2428.197,y=200.437),Point(x=2560.0,y=335.419),
                Point(x=2557.386,y=503.165),Point(x=2320.596,y=503.103),Point(x=2156.083, y=628.943),Point(x=2161.111,y=785.519),
                Point(x=2002.115, y=894.647),Point(x=1838.456,y=877.874),Point(x=1436.53,y=874.636),Point(x=1411.403,y=758.579),
                Point(x=1353.853,y=751.74),Point(x=1345.264, y=453.461),Point(x=1426.011,y=421.129)]
    ),
)


# NDJSON

polygon_prediction_ndjson = {
  'name': 'polygon',
  'confidence': 0.5,
  'polygon': [
    {'x': 1489.581, 'y': 183.934},
    {'x': 2278.306, 'y': 256.885},
    {'x': 2428.197, 'y': 200.437},
    {'x': 2560.0, 'y': 335.419},
    {'x': 2557.386, 'y': 503.165},
    {'x': 2320.596, 'y': 503.103},
    {'x': 2156.083, 'y': 628.943},
    {'x': 2161.111, 'y': 785.519},
    {'x': 2002.115, 'y': 894.647},
    {'x': 1838.456, 'y': 877.874},
    {'x': 1436.53, 'y': 874.636},
    {'x': 1411.403, 'y': 758.579},
    {'x': 1353.853, 'y': 751.74},
    {'x': 1345.264, 'y': 453.461},
    {'x': 1426.011, 'y': 421.129},
    {'x': 1489.581, 'y': 183.934}
  ]
}

In [636]:
####### Free text #######
# Confidence is not supported for text prediction
# Python annotation
text_annotation = ClassificationAnnotation(
  name="free_text",  # must match your ontology feature's name
  value=Text(answer="sample text")
)

# NDJSON
text_annotation_ndjson = {
  'name': 'free_text',
  'answer': 'sample text',
}

In [637]:
######### Segmentation mask #########

# Python 
# Identifying what values in the numpy array correspond to the mask annotation
color = (0, 0, 0)

# convert a polygon to mask
im_height, im_width = 100,100 #need to provide the height and width of image.
mask_data = MaskData(arr=
                     polygon_prediction.value.draw(height=im_height,width=im_width,color=color))

# convert a 2D array to 3D array
arr_2d = np.zeros((100,100), dtype='uint8')
mask_data = MaskData.from_2D_arr(arr_2d)

# a 3D array where 3rd axis is RGB values.
mask_data = MaskData(arr= np.zeros([400,450,3],dtype='uint8'))

mask_prediction = ObjectAnnotation(
  name = "mask", # must match your ontology feature's name
  confidence=0.5,
  value=Mask(mask=mask_data, color=color),
)


# NDJSON
mask_prediction_ndjson = {
  'name': 'mask',
  'confidence': 0.5,
  'classifications': [],
  'mask': {'instanceURI': 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA',
  'colorRGB': (0, 0, 0)}
}



In [638]:
######## Point ########

# Python Annotation
point_prediction = ObjectAnnotation(
  name = "point",  # must match your ontology feature's name
  confidence=0.5,
  value = Point(x=1166.606, y=1441.768),
)


# NDJSON
point_prediction_ndjson = {
  'name': 'point',
  'confidence': 0.5,
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

In [639]:
###### Polyline ######


# Python Annotation 

polyline_prediction = ObjectAnnotation(
  name = "polyline", # must match your ontology feature's name
  confidence=0.5, ## Not supported for python annotation tools
  value=Line( # Coordinates for the keypoints in your polyline
        points=[Point(x=2534.353, y=249.471),Point(x=2429.492, y=182.092),Point(x=2294.322, y=221.962),Point(x=2224.491, y=180.463),Point(x=2136.123, y=204.716),
                Point(x=1712.247, y=173.949),Point(x=1703.838, y=84.438),Point(x=1579.772, y=82.61),Point(x=1583.442, y=167.552),
                Point(x=1478.869, y=164.903),Point(x=1418.941, y=318.149),Point(x=1243.128, y=400.815),Point(x=1022.067, y=319.007),
                Point(x=892.367, y=379.216),Point(x=670.273, y=364.408),Point(x=613.114, y=288.16),Point(x=377.559, y=238.251),
                Point(x=368.087, y=185.064),Point(x=246.557, y=167.286),Point(x=236.648, y=285.61),Point(x=90.929, y=326.412)]
    ),
)

# NDJSON
polyline_prediction_ndjson = {
  'name': 'polyline',
  'confidence':0.5,
  'classifications': [],
  'line': [
    {'x': 2534.353, 'y': 249.471},
    {'x': 2429.492, 'y': 182.092},
    {'x': 2294.322, 'y': 221.962},
    {'x': 2224.491, 'y': 180.463},
    {'x': 2136.123, 'y': 204.716},
    {'x': 1712.247, 'y': 173.949},
    {'x': 1703.838, 'y': 84.438},
    {'x': 1579.772, 'y': 82.61},
    {'x': 1583.442, 'y': 167.552},
    {'x': 1478.869, 'y': 164.903},
    {'x': 1418.941, 'y': 318.149},
    {'x': 1243.128, 'y': 400.815},
    {'x': 1022.067, 'y': 319.007},
    {'x': 892.367, 'y': 379.216},
    {'x': 670.273, 'y': 364.408},
    {'x': 613.114, 'y': 288.16},
    {'x': 377.559, 'y': 238.251},
    {'x': 368.087, 'y': 185.064},
    {'x': 246.557, 'y': 167.286},
    {'x': 236.648, 'y': 285.61},
    {'x': 90.929, 'y': 326.412}
  ]
}


## Step 1: Import data rows into Catalog

In [640]:
# send a sample image as batch to the project
test_img_url = {
    "row_data": "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg",
    "global_key": str(uuid.uuid4())
}
dataset = client.create_dataset(name="image_prediction_demo")
data_row = dataset.create_data_row(test_img_url)
print(data_row)

<DataRow {
    "created_at": "2023-01-23 22:38:32+00:00",
    "external_id": null,
    "global_key": "3fd7d61a-5aba-47c1-afba-12498818c7b2",
    "media_attributes": {},
    "metadata": [],
    "metadata_fields": [],
    "row_data": "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg",
    "uid": "cld9e3z430mgy072abell3l7a",
    "updated_at": "2023-01-23 22:38:32+00:00"
}>


## Step 2: Create/select an Ontology for your model predictions
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the name/instructions fields in your annotations to ensure the correct feature schemas are matched.


In [641]:
ontology_builder = OntologyBuilder(
  classifications=[ # List of Classification objects
    Classification( # Radio classification given the name "text" with two options: "first_radio_answer" and "second_radio_answer"
      class_type=Classification.Type.RADIO, 
      instructions="radio_question", 
      options=[
        Option(value="first_radio_answer"),
        Option(value="second_radio_answer")
      ]
    ),
    Classification( # Checklist classification given the name "text" with two options: "first_checklist_answer" and "second_checklist_answer"
      class_type=Classification.Type.CHECKLIST, 
      instructions="checklist_question", 
      options=[
        Option(value="first_checklist_answer"),
        Option(value="second_checklist_answer")            
      ]
    ), 
    Classification( # Text classification given the name "text"
      class_type=Classification.Type.TEXT,
      instructions="free_text"
    ),
    Classification(
        class_type=Classification.Type.RADIO, 
        instructions="nested_radio_question",
        options=[
            Option("first_radio_answer",
                options=[
                    Classification(
                        class_type=Classification.Type.RADIO,
                        instructions="sub_radio_question",
                        options=[Option("first_sub_radio_answer")]
                    )
                ]
            )
          ] 
        ),
    Classification(
      class_type=Classification.Type.CHECKLIST, 
      instructions="nested_checklist_question",
      options=[
          Option("first_checklist_answer",
            options=[
              Classification(
                  class_type=Classification.Type.CHECKLIST, 
                  instructions="sub_checklist_question", 
                  options=[Option("first_sub_checklist_answer")]
              )
          ]
        )
      ]
    ),      
  ],
  tools=[ # List of Tool objects
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX, 
      name="bounding_box"), 
    Tool( # Bounding Box tool given the name "box"
      tool=Tool.Type.BBOX, 
      name="bbox_with_radio_subclass",
      classifications=[
            Classification(
                class_type=Classification.Type.RADIO,
                instructions="sub_radio_question",
                options=[
                  Option(value="first_sub_radio_answer")
                ]
              ),
        ]
      ), 
    Tool( # Polygon tool given the name "polygon"
      tool=Tool.Type.POLYGON, 
      name="polygon"),
    Tool( # Segmentation mask tool given the name "mask"
      tool=Tool.Type.SEGMENTATION, 
      name="mask"),
 	  Tool( # Point tool given the name "point"
      tool=Tool.Type.POINT, 
      name="point"), 
    Tool( # Polyline tool given the name "line"
      tool=Tool.Type.LINE, 
      name="polyline")]
)

ontology = client.create_ontology("Image Prediction Import Demo", ontology_builder.asdict(), media_type=MediaType.Image)

## Step 3: Create a Model and Model Run

In [642]:
# create Model
model = client.create_model(name="image_model_run_" + str(uuid.uuid4()),
                            ontology_id=ontology.uid)
# create Model Run
model_run = model.create_model_run("iteration 1")

## Step 4: Send data rows to the Model Run

In [643]:
model_run.upsert_data_rows([data_row.uid])

True

## Step 5. Create the predictions payload

Create the annotations payload using the snippets of code in [Supported python annotation types and NDJSON](https://colab.research.google.com/drive/14ZdYPz8s58D07AOUjKuJ54rFjM0JVNlf#scrollTo=OePiibbed0nG&line=1&uniqifier=1).

The resulting label_ndjson should have exactly the same content for annotations that are supported by both (with exception of the uuid strings that are generated)

In [644]:
# Create a Label for predictions
label_prediction = Label(
    data=ImageData(uid=data_row.uid),
    annotations = [
      radio_prediction,
      checklist_prediction, 
      bbox_prediction, 
      bbox_with_radio_subclass_prediction, 
      polygon_prediction, 
      mask_prediction, 
      point_prediction,
      nested_radio_prediction
      ]
)

# Create a label list 
label_list_prediction = [label_prediction]

# Convert the prediction label from a Labelbox class object to the underlying NDJSON format required for upload - uploads can be directly built in this syntax as well
ndjson_prediction = list(NDJsonConverter.serialize(label_list_prediction))

If using NDJSON

In [645]:

ndjson_prediction_method2 = []
for annot in [
    radio_prediction_ndjson,
    checklist_prediction_ndjson, 
    bbox_prediction_ndjson, 
    bbox_with_radio_subclass_prediction_ndjson, 
    polygon_prediction_ndjson, 
    mask_prediction_ndjson, 
    point_prediction_ndjson,
    polyline_prediction_ndjson,
    text_annotation_ndjson, 
    nested_radio_prediction_ndjson,
    nested_checklist_prediction_ndjson
    
]:
  annot.update({
      'uuid': str(uuid.uuid4()),
      'dataRow': {'id': data_row.uid},
  })
  ndjson_prediction_method2.append(annot)

## Step 6. Upload the predictions payload to the Model Run 

In [646]:
# Upload the prediction label to the Model Run
upload_job_prediction = model_run.add_predictions(
    name="prediction_upload_job"+str(uuid.uuid4()),
    predictions=ndjson_prediction_method2)

# Errors will appear for annotation uploads that failed.
print("Errors:",  upload_job_prediction.errors)

Errors: []


## Step 7: Send annotations to a model run
To visualize both annotations and predictions in the model run we will create a project with ground truth annotations. 
To send annotations to a Model Run, we must first import them into a project, create a label payload and then send them to the Model Run.

##### 7.1. Create a labelbox project

In [647]:
# Create a Labelbox project
project = client.create_project(name="image_prediction_demo",                                    
                                    queue_mode=QueueMode.Batch,
                                    # Quality Settings setup 
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=MediaType.Image)
project.setup_editor(ontology)

##### 7.2. Create a batch to send to the project 

In [648]:
project.create_batch(
  "batch_predictions_demo", # Each batch in a project must have a unique name
  dataset.export_data_rows(), # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

<Batch ID: b5633900-9b6e-11ed-bd27-9dc8c0fce61f>

##### 7.3 Create the annotations payload

In [649]:
########### Annotations ###########
radio_annotation_ndjson = {
  'name': 'radio_question',
  'answer': {'name': 'second_radio_answer'}
} 

nested_radio_annotation_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer"},
  "classifications" : [
   {'name': 'sub_radio_question', 'answer': {'name': 'first_sub_radio_answer'}}
   ]
}

checklist_annotation_ndjson = {
  'name': 'checklist_question',
  'answer': [
    {'name': 'first_checklist_answer'},
    {'name': 'second_checklist_answer'}
  ]
}

bbox_annotation_ndjson = {
  'name': 'bounding_box',
  'bbox': {
          "top": 977,
          "left": 1690,
          "height": 330,
          "width": 225
      }
}

bbox_with_radio_subclass_ndjson = {
    "name": "bbox_with_radio_subclass", 
    "classifications": [{
        "name": "sub_radio_question",
        "answer": 
            { "name":"first_sub_radio_answer" }
         
    }],
    "bbox": {
          "top": 933,
          "left": 541,
          "height": 191,
          "width": 330
        }
}

polygon_annotation_ndjson = {
  'name': 'polygon',
  'polygon': [
    {'x': 1489.581, 'y': 183.934},
    {'x': 2278.306, 'y': 256.885},
    {'x': 2428.197, 'y': 200.437},
    {'x': 2560.0, 'y': 335.419},
    {'x': 2557.386, 'y': 503.165},
    {'x': 2320.596, 'y': 503.103},
    {'x': 2156.083, 'y': 628.943},
    {'x': 2161.111, 'y': 785.519},
    {'x': 2002.115, 'y': 894.647},
    {'x': 1838.456, 'y': 877.874},
    {'x': 1436.53, 'y': 874.636},
    {'x': 1411.403, 'y': 758.579},
    {'x': 1353.853, 'y': 751.74},
    {'x': 1345.264, 'y': 453.461},
    {'x': 1426.011, 'y': 421.129},
    {'x': 1489.581, 'y': 183.934}
  ]
}

mask_annotation_ndjson = {
  'name': 'mask',
  'classifications': [],
  'mask': {'instanceURI': 'https://storage.labelbox.com/cjhfn5y6s0pk507024nz1ocys%2F1d60856c-59b7-3060-2754-83f7e93e0d01-1?Expires=1666901963361&KeyName=labelbox-assets-key-3&Signature=t-2s2DB4YjFuWEFak0wxYqfBfZA',
  'colorRGB': (0, 0, 0)}
}


point_annotation_ndjson = {
  'name': 'point',
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

point_annotation_ndjson = {
  'name': 'point',
  'classifications': [],
  'point': {'x': 1166.606, 'y': 1441.768}
}

polyline_annotation_ndjson = {
  'name': 'polyline',
  'classifications': [],
  'line': [
    {'x': 2534.353, 'y': 249.471},
    {'x': 2429.492, 'y': 182.092},
    {'x': 2294.322, 'y': 221.962},
    {'x': 2224.491, 'y': 180.463},
    {'x': 2136.123, 'y': 204.716},
    {'x': 1712.247, 'y': 173.949},
    {'x': 1703.838, 'y': 84.438},
    {'x': 1579.772, 'y': 82.61},
    {'x': 1583.442, 'y': 167.552},
    {'x': 1478.869, 'y': 164.903},
    {'x': 1418.941, 'y': 318.149},
    {'x': 1243.128, 'y': 400.815},
    {'x': 1022.067, 'y': 319.007},
    {'x': 892.367, 'y': 379.216},
    {'x': 670.273, 'y': 364.408},
    {'x': 613.114, 'y': 288.16},
    {'x': 377.559, 'y': 238.251},
    {'x': 368.087, 'y': 185.064},
    {'x': 246.557, 'y': 167.286},
    {'x': 236.648, 'y': 285.61},
    {'x': 90.929, 'y': 326.412}
  ]
}

nested_checklist_prediction_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer", 
      "classifications" : [
        {
          "name": "sub_checklist_question", 
          "answer": {"name": "first_sub_checklist_answer"}
        }          
      ]         
  }]
}


##### 7.4. Create the label object

In [650]:
# Create a Label object by identifying the applicavle data row in Labelbox and providing a list of annotations
ndjson_annotation = []
for annot in [
    radio_annotation_ndjson, 
    checklist_annotation_ndjson, 
    bbox_annotation_ndjson, 
    bbox_with_radio_subclass_ndjson, 
    polygon_annotation_ndjson, 
    mask_annotation_ndjson, 
    point_annotation_ndjson, 
    polyline_annotation_ndjson,
    nested_radio_annotation_ndjson,
    nested_checklist_prediction_ndjson
]:
  annot.update({
      'uuid': str(uuid.uuid4()),
      'dataRow': {'id': data_row.uid},
  })
  ndjson_annotation.append(annot) 



##### 7.5. Upload annotations to the project using Label Import

In [651]:
upload_job_annotation = LabelImport.create_from_objects(
    client = client,
    project_id = project.uid,
    name="annotation_import_" + str(uuid.uuid4()),
    labels=ndjson_annotation)

upload_job_annotation.wait_until_done()
# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_annotation.errors)


Errors: []


##### 7.6 Send the annotations to the Model Run

In [652]:
# get the labels id from the project
label_ids = [x['ID'] for x in project.export_labels(download=True)]
model_run.upsert_labels(label_ids)

True

## Optional deletions for cleanup 


In [653]:
#upload_job
# project.delete()
# dataset.delete()